<a href="https://colab.research.google.com/github/wayamhui/ISYS5002_Assignment/blob/main/Assignment20486978.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment20486978.ipynb

In [1]:
# install pip and import useful modules

!pip install kora -q
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd



/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import pandas as pd

In [22]:
# Tax calculation function standby in here, and it is used in the add_tax_in_records function

def tax_cal(annualincome):
  if annualincome <=18200:
    tax_payable = 0
  elif annualincome <=45000:
    tax_payable = ( annualincome - 18200 ) * 0.19 
  elif annualincome <=120000:
    tax_payable = ( annualincome - 45000 ) * 0.325 + 5092
  elif annualincome <=180000:
    tax_payable = ( annualincome - 120000 ) * 0.37 + 29467
  else:
    tax_payable = ( annualincome - 180000 ) * 0.45 + 51667
  return tax_payable


In [23]:
# URL creation function ready

# user input ticker symbol


def build_url(ticker_symbol):

  # print(ticker_symbol)
  # user tikcer to get profile page
  example_url = 'https://finance.yahoo.com/quote/{}/profile?p={}'
  result = example_url.format (ticker_symbol,ticker_symbol)
  return result

In [24]:
# Validate the input for ticker

def validate_url_of_ticker(ticker):
  valid = False   # F needs to be capitalized

  while not valid:
    try:
      url = build_url(ticker)
      # print(url)
      wd.get(url)    

      soup = BeautifulSoup(wd.page_source, "html.parser")

      table = soup.find('table')
      body = soup.find('tbody')
      rows = body.find_all('tr')
      
      valid = True

    except AttributeError:
      print("The ticker inputted seems not a valid company, please re-input ticker symbol for a comany...") 
      ticker = input ("Please input a ticker symbol to view the excecutives' annual pay and the calculated AXO tax (e.g. NAB.AX): ")
  return url


In [25]:
def get_rows_from_url_table():

  wd.get(url)
  soup = BeautifulSoup(wd.page_source, "html.parser")
  table = soup.find('table')
  body = soup.find_all('tbody')

  # print (body)
  # print (len(body))

  body = soup.find('tbody')

  # print (body)
  # print (len(body))

  rows = body.find_all('tr')
  # print (len(rows))

  row1 = rows[0]
  cells = row.find_all('td')
  # print (len(cells))

  name = cells[0]
  title = cells[1]
  pay = cells[2]

  return rows

# rows = get_rows_from_url_table()

def get3cells(row):    
  cells = row.find_all('td')
  # print (len(cells))
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  
  # for item in [0,1,2,3,4]:
  #  print (cells[item].text)  # to test print 4 cells together.

  return {'name': name, 'title': title, 'pay': pay}  # using dictionary type, so no need to remember zero refers to name, 1 refers to title, etc)

# print (get3cells(rows[3])) # hange to commentary, after tested

NameError: ignored

In [17]:
# get records from the URL's profile table
def get_exec_records():
  exec_records = []
  for row in rows:
    result = get3cells(row)
    exec_records.append(result)
  return exec_records

# print (get_exec_records())

In [18]:
# validate foreign exchange rate

def validate_fx():
  valid = False
  while not valid:
    try: 
      fx = input('If the compnay of the ticker is not listed in Australia, please input FX: 1 ForeignCurrency = ? AUD: (enter blank if Australia listed) ')
      if fx == "":
        fx = 1.0    
      else:
        fx = float(fx)
      valid = True
    except ValueError:
      print ('please re-input FX in number format (decimal points are accepted...')
  return fx

validate_fx()

If the compnay of the ticker is not listed in Australia, please input FX: 1 ForeignCurrency = ? AUD: (enter blank if Australia listed) 1


1.0

In [19]:
def thousand_translate(pay_display):
  last_char = pay_display[-1]
  last3_char = pay_display[-3:-1] + pay_display[-1] 

  if last3_char == "N/A":
    salary = -1     # to distignuish N/A from zero salary
    
  elif last_char == 'M':    
    number = pay_display[0:-1]
    salary = float(number) * 1000000
  elif last_char == 'k': 
    number = pay_display[0:-1]
    salary = float(number) * 1000  
  else:
    salary = float(pay)  # some executives can have salary of $1 (i.e. not ending with k or M)
  return salary

records = get_exec_records()
# pay = records[2]['pay']
# print (thousand_translate(pay))

NameError: ignored

In [12]:
def add_tax_in_records(fx):
  for record in records:
    income = record['pay']
    
    temp = thousand_translate(income)
    
    if temp == -1:
      record['tax'] = 'N/A'
    
    else:
      temp1 = tax_cal(temp * fx)
      if temp1 > 1000000:         # this if phrase is to convert big amounts to "M" or "k" format with 2 decimal point that is consistnent with pay
        temp2 = str(round(temp1/1000000, 2)) +"M"
      elif temp1 > 1000:
        temp2 = str(round(temp1/1000, 2)) +"k"
      else:
        temp2 = str(temp1)
      record['tax'] = temp2
      # print (record['tax'])  # test print before finalizing the function
  return temp2  # it is also okay not to return any value, now returns the last non-N/A tax record
    
# add_tax_in_records(validate_fx())

In [13]:
def output_to_cvs(name):
  
  fullname = '/content/' + name + '.csv'  
  with open(fullname,'w') as f:
    f.writelines("Name, Title, Pay(Original_Currency), Tax(AUD)")
    f.writelines("\n")
    for record in records:
      temp3 = str(record['name'])+ "," + str(record['title']) + "," + str(record['pay']) + "," + str(record['tax']) 
      f.writelines(temp3)
      f.writelines("\n")  
  f.close()


# This session is the main program by recalling the functions defined in above. 
This session can be run multiple times, all previous session only need to run once. 

In [21]:
# main input:
ticker = input ("Please input a ticker symbol to view the excecutives' annual pay and the calculated AXO tax (e.g. NAB.AX): ")

# functions used:
validate_url_of_ticker(ticker)
get_exec_records()
fx = validate_fx()
add_tax_in_records(fx)

# Ask for csv fileame to use for the output:

filename = input('please input the filename you would like to save as a csv file (enter blank to use the first 3 charactors of the ticker as filename): ')
if filename == '':
  output_to_cvs(ticker[0:3])  # use first 3 charactors to avoid picking the dot for use in filename
else:
  output_to_cvs(filename) 




Please input a ticker symbol to view the excecutives' annual pay and the calculated AXO tax (e.g. NAB.AX): nab.ax


NameError: ignored